In [1]:
import numpy as np
import math


In [2]:
def polymer_surface_plane (polymer_atoms):
    
    polymer_plane_constant = math.ceil(max(polymer_atoms [:,:,2]))
    
    return polymer_plane_constant


In [3]:
def perpendicular_distance (center_of_mass_traj, total_frame, polymer_plane_constant = 44):
    
    perpendicular_distance = np.empty((0,1), dtype=float, order='C')
    
    for frame_num in total_frame:

        drug_com_xyz = center_of_mass_traj[frame_num]
        
        drug_com_dist = drug_com_xyz[2] - polymer_plane_constant
        
        perpendicular_distance = np.append(perpendicular_distance, drug_com_dist)
        
    return perpendicular_distance




In [4]:
def calc_center_of_mass (atomic_mass, xcoords, ycoords, zcoords):
    
    xproduct = 0.0
    yproduct = 0.0
    zproduct = 0.0

    for i in np.arange(len(atomic_mass)):
        
        xproduct = xproduct + atomic_mass[i] * xcoords[i]
        yproduct = yproduct + atomic_mass[i] * ycoords[i]
        zproduct = zproduct + atomic_mass[i] * zcoords[i]
        
    return xproduct/sum(atomic_mass), yproduct/sum(atomic_mass), zproduct/sum(atomic_mass)




In [5]:
def within_dist_query (atom1_traj, atom2_traj, total_frame, threshold):
    
    """
        Calculate the time frames where atom1 and atom2 distance is within threshold
        
    Parameters
    ----------
    atom1_traj : 3D Coordinates of atom1
    
    atom2_traj : 3D Coordinates of atom2
    
    total_frame : Time frames

    Returns
    -------
    
    within_dist_timestep : Timesteps where exists an instance of distance within threshold
    
    within_dist_atom_index : Index of atom of atom2 in every timestep that is within threshold distance of atoms of atom1_traj
                        If there is multiple atoms of atom2_traj meet the threshold criteria, the closest atom index is returned.
    """
    
    within_dist_timestep = np.empty((0,1), dtype=int, order='C')
    
    within_dist_atom_index = np.empty((0,1), dtype=int, order='C')
            
    for frame_num in total_frame:
        
        atom1_traj_xyz = atom1_traj[frame_num]
        
        atom2_traj_xyz = atom2_traj[frame_num]
    
        atom2_traj_x, atom2_traj_y, atom2_traj_z = split_xyz (atom2_traj_xyz)
    
        dist_atom1_atom2,_ , _, _ = distance_periodicity (atom1_traj_xyz[0], atom1_traj_xyz[1],atom1_traj_xyz[2], atom2_traj_x, atom2_traj_y, atom2_traj_z)

    
        if min(dist_atom1_atom2) <= threshold :
            
            index = np.argmin(dist_atom1_atom2) 
            
            within_dist_atom_index = np.append(within_dist_atom_index, index)
        
            within_dist_timestep = np.append(within_dist_timestep, frame_num)
            
    return within_dist_timestep, within_dist_atom_index


# =============================================================================
# def donor_acceptor_dist (atom1_traj, atom2_traj, total_frame, hbond_threshold = 3.5):
#     
#     """
#         Calculate the time frames and donor atoms where donor acceptor distance is within 3.5A
#         
#     Parameters
#     ----------
#     atom1_traj : 3D Coordinates of drug acceptor
#     
#     atom2_traj : 3D Coordinates of Polymer Donor
#     
#     total_frame : Timesteps from either perpendicular distance or interpolation
# 
#     Returns
#     -------
#     
#     donor_acceptor_timestep : Timesteps where exists an instance of donor-acceptance which are within 3.5 A
#     
#     donor_atom_index : Index of donor (nh) atoms in every timestep that is within 3.5 A of acceptor
#                         If there is multiple donor atoms that met the criteria, the closest donor atom is returned.
#     """
#     
#     donor_acceptor_timestep = np.empty((0,1), dtype=int, order='C')
#     
#     donor_atom_index = np.empty((0,1), dtype=int, order='C')
#             
#     for frame_num in total_frame:
#         
#         atom1_traj_xyz = atom1_traj[frame_num]
#         
#         atom2_traj_xyz = atom2_traj[frame_num]
#     
#         atom2_traj_x, atom2_traj_y, atom2_traj_z = split_xyz (atom2_traj_xyz)
#     
#         donor_acceptor_dist,_ , _, _ = distance_periodicity (atom1_traj_xyz[0], atom1_traj_xyz[1],atom1_traj_xyz[2], atom2_traj_x, atom2_traj_y, atom2_traj_z)
# 
#     
#         if min(donor_acceptor_dist) <= hbond_threshold :
#             
#             index = np.argmin(donor_acceptor_dist) 
#             donor_atom_index = np.append(donor_atom_index, index)
#         
#             donor_acceptor_timestep = np.append(donor_acceptor_timestep, frame_num)
#             
#     return donor_acceptor_timestep, donor_atom_index
# =============================================================================



In [6]:
def pairwise_distance_periodicity (X1,Y1,Z1,X2,Y2,Z2):
    """
    Apply periodicity

    Parameters
    ----------
    X1 : Numpy 1D array
        x axis coordinates of either atom 1 trajectory or all atoms of molecule 1
    Y1 : Numpy 1D array
        y axis coordinates of either atom 1 trajectory or all atoms of molecule 1
    Z1 : Numpy 1D array
        z axis coordinates of either atom 1 trajectory or all atoms of molecule 1
    X2 : Numpy 1D array
        x axis coordinates of either atom 2 trajectory or all atoms of molecule 2
    Y2 : Numpy 1D array
        y axis coordinates of either atom 2 trajectory or all atoms of molecule 2
    Z2 : Numpy 1D array
        z axis coordinates of either atom 2 trajectory or all atoms of molecule 2

    Returns
    -------
    distance: Numpy 2D array
            Euclidean pairwise distance of (X1,Y1,Z1) and (X2,Y2,Z2)

    """

    x_coord_diff = abs(X1 [np.newaxis,:] - X2[:,np.newaxis])

    for i in range(len(x_coord_diff)):
        for j in range(len(x_coord_diff[i])):
            if x_coord_diff[i][j] > 36:
                x_coord_diff[i][j] = abs(x_coord_diff[i][j] - 72)

    y_coord_diff = abs(Y1 [np.newaxis,:] - Y2[:,np.newaxis])

    for i in range(len(y_coord_diff)):
        for j in range(len(y_coord_diff[i])):
            if y_coord_diff[i][j] > 36:
                y_coord_diff[i][j] = abs(y_coord_diff[i][j] - 72)
        
    z_coord_diff = abs(Z1[np.newaxis,:] - Z2[:,np.newaxis])

    distance = np.sqrt(np.square(x_coord_diff) + np.square(y_coord_diff) + np.square(z_coord_diff))
    
    return distance




In [7]:
def split_xyz (coords):
    """
    Parameters
    ----------
    coords : 3D Coordinates

    Returns
    -------
    Coordinates on each axis
 
    """
    # Calculate the x coordinates
    X1 = coords[:,0]

    # Calculate the y coordinates
    Y1 = coords[:,1]

    # Calculate the z coordinates
    Z1 = coords[:,2]
    
    return X1, Y1, Z1



In [8]:
def distance_periodicity (X1,Y1,Z1,X2,Y2,Z2):
    
    """
    Apply periodic boundaries
    
    Periodic Boundaries in X and Y direction is 72 A.
    Z axis does not have any periodic boundary
    
    So, the periodicity conditions

    Distance in X axis = if |x2 − x1|> 36:
                                72 −|x2 − x1|
                            
                         else:
                                x2 − x1
    
    Distance in Y axis = same as X
    
    Euclidean Shortest Distance = √(〖"(Distance in X axis)" 〗^2+〖"(Distance in Y axis)" 〗^2+〖"(Distance in Z axis)" 〗^2 )

    Parameters
    ----------
    X1 : Numpy 1D array
        x axis coordinates of atom 1 trajectory
    Y1 : Numpy 1D array
        y axis coordinates of atom 1 trajectory 
    Z1 : Numpy 1D array
        z axis coordinates of atom 1 trajectory 
    X2 : Numpy 1D array
        x axis coordinates of atom 2 trajectory 
    Y2 : Numpy 1D array
        y axis coordinates of atom 2 trajectory 
    Z2 : Numpy 1D array
        z axis coordinates of atom 2 trajectory 

    Returns
    -------
    distance: Numpy 1D array
            Euclidean distance of (X1,Y1,Z1) and (X2,Y2,Z2)

    """

    x_coord_diff = np.array([(72.475-abs(item)) if abs(item)>36.2375 else item for item in (X1-X2)])
    
    y_coord_diff = np.array([(72.475-abs(item)) if abs(item)>36.2375 else item for item in (Y1-Y2)])
        
    z_coord_diff = np.array((Z1 - Z2))

    euclidean_distance = np.sqrt(np.square(x_coord_diff) + np.square(y_coord_diff) + np.square(z_coord_diff))
    
    return euclidean_distance, x_coord_diff, y_coord_diff, z_coord_diff




In [9]:
def apply_rdp_compression (coords, epsilon = 1):
    """
    

    Parameters
    ----------
    coords : 1D, 2D or 3D coordinates
        DESCRIPTION.
    epsilon : Float
        default 1.

    Returns
    -------
    None.

    """
    
    from rdp import rdp
    
    mask = rdp(coords, epsilon, return_mask=True)
    mask_array = np.arange(len(mask))
    compressed_coords = coords[mask]
    compressed_time_frames = mask_array [mask]
    
    return compressed_coords, compressed_time_frames


import gzip




In [10]:
def modified_split_multimol2(mol2_path,timeframe):
    r"""
    Generator function that
    splits a multi-mol2 file into individual Mol2 file contents.
    Parameters
    -----------
    mol2_path : str
      Path to the multi-mol2 file. Parses gzip files if the filepath
      ends on .gz.
    Returns
    -----------
    A generator object for lists for every extracted mol2-file. Lists contain
        the molecule ID and the mol2 file contents.
        e.g., ['ID1234', ['@<TRIPOS>MOLECULE\n', '...']]. Note that bytestrings
        are returned (for reasons of efficieny) if the Mol2 content is read
        from a gzip (.gz) file.
    """
    if mol2_path.endswith('.mol2'):
        open_file = open
        read_mode = 'r'
    elif mol2_path.endswith('mol2.gz'):
        open_file = gzip.open
        read_mode = 'rb'
    else:
        raise ValueError('Wrong file format;'
                         'allowed file formats are .mol2 and .mol2.gz.')

    check = {'rb': b'@<TRIPOS>MOLECULE', 'r': '@<TRIPOS>MOLECULE'}

    with open_file(mol2_path, read_mode) as f:
        mol2 = ['', []]
        i = 0
        while i <= timeframe:
            i += 1
            try:
                line = next(f)
                if line.startswith(check[read_mode]):
                    if mol2[0]:
                        yield(mol2)
                    mol2 = ['', []]
                    mol2_id = next(f)
                    mol2[0] = mol2_id.rstrip()
                    mol2[1].append(line)
                    mol2[1].append(mol2_id)
                else:
                    mol2[1].append(line)
                
            except StopIteration:
                yield(mol2)
                return
            
